# Abstracted Form 

In [62]:
from get_files import get_super_table
import time
import sys

In [60]:
tic = time.perf_counter()
df = get_super_table()
toc = time.perf_counter()
print(f"Got the super table in {toc - tic:0.4f} seconds")

Got the super table in 2.4181 seconds


In [64]:
sys.getsizeof(df) / (10**9)

0.532076376

In [4]:
from helper import get_req_handler, initBoto3Session, get_s3_objs
from dotenv import load_dotenv
import boto3
import os
from io import StringIO

import pandas as pd
import numpy as np

# Set Up 

In [5]:
load_dotenv()

aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"]
aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"]

client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

bucket_name = os.environ["BUCKET_NAME"]
weather_dir = os.environ["WEATHER_DIR"]
bing_dir = os.environ["BING_DIR"]

In [6]:
def get_csv(file_name, data_dir):
    csv_obj = client.get_object(Bucket=bucket_name, Key= data_dir + file_name)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string))
    df = df.drop("Unnamed: 0", axis = 1)
    return df

# Camera to Location Map 

In [7]:
cam_to_loc = df = get_csv("camera_station_mapping.csv", "").rename(
    columns = {"rainfall" : "rainfall_station_id",
               "not_rainfall" : "non_rainfall_station_id",
               "region" : "2hr_forecast_area"})
cam_to_loc.head()

,cam_id,non_rainfall_station_id,rainfall_station_id,2hr_forecast_area,compass
0,1001,S108,S119,Kallang,south
1,1002,S108,S215,Geylang,east
2,1003,S108,S123,Kallang,central
3,1004,S108,S215,Kallang,central
4,1005,S107,S221,Paya Lebar,east


# Read Files 

IMPORTANT: Note all data will be consistent (sometime the stations wont appear)  
Time stamps are also different - shimar looking into it 

In [8]:
rainfall_realtime = get_csv("rainfall-realtime.csv", weather_dir)
rainfall_realtime.head()

,timestamp,station_id,station_loc,station_name,rainfall_realtime,call_timestamp
0,2022-03-31T222500,S77,"(1.2937, 103.8125)",Alexandra Road,0.0,2022-03-31T223549
1,2022-03-31T222500,S109,"(1.3764, 103.8492)",Ang Mo Kio Avenue 5,0.0,2022-03-31T223549
2,2022-03-31T222500,S90,"(1.3191, 103.8191)",Bukit Timah Road,0.0,2022-03-31T223549
3,2022-03-31T222500,S114,"(1.38, 103.73)",Choa Chu Kang Avenue 4,0.0,2022-03-31T223549
4,2022-03-31T222500,S50,"(1.3337, 103.7768)",Clementi Road,0.0,2022-03-31T223549


In [9]:
non_rainfall_realtime = get_csv("non-rainfall-realtime.csv", weather_dir)
non_rainfall_realtime.head()

,timestamp,station_id,station_loc,station_name,wind_speed_realtime,wind_dir_realtime,RH%_realtime,air_temp_realtime,call_timestamp
0,2022-03-31T223000,S107,"(1.3135, 103.9625)",East Coast Parkway,2.7,251,78.2,28.6,2022-03-31T223549
1,2022-03-31T223000,S108,"(1.2799, 103.8703)",Marina Gardens Drive,1.6,251,93.9,27.6,2022-03-31T223549
2,2022-03-31T223000,S44,"(1.34583, 103.68166)",Nanyang Avenue,1.1,230,82.7,27.4,2022-03-31T223549
3,2022-03-31T223000,S106,"(1.4168, 103.9673)",Pulau Ubin,0.9,266,96.5,25.7,2022-03-31T223549
4,2022-03-31T223000,S24,"(1.3678, 103.9826)",Upper Changi Road North,1.8,329,84.7,27.5,2022-03-31T223549


In [10]:
forecast_4d = get_csv("forecast-4DAY.csv", weather_dir)
forecast_4d.head()

,4day_date,4day_forecast,4day_temperature_low,4day_temperature_high,4day_relative_humidity_low,4day_relative_humidity_high,4day_wind_speed_low,4day_wind_speed_high,4day_wind_direction,4day_update_timestamp,call_timestamp
0,2022-04-01T000000,Afternoon thundery showers,24,34,65,95,5,15,NW,2022-03-31T173213,2022-03-31T223549
1,2022-04-02T000000,Afternoon thundery showers,24,34,65,95,5,15,NW,2022-03-31T173213,2022-03-31T223549
2,2022-04-03T000000,Afternoon thundery showers,24,34,65,90,5,15,W,2022-03-31T173213,2022-03-31T223549
3,2022-04-04T000000,Afternoon thundery showers,24,34,65,90,5,15,W,2022-03-31T173213,2022-03-31T223549
4,2022-04-01T000000,Afternoon thundery showers,24,34,65,95,5,15,NW,2022-03-31T173213,2022-03-31T223627


In [11]:
forecast_24h = get_csv("forecast-24HR.csv", weather_dir)
forecast_24h.head()

,24hr_start,24hr_end,24hr_general_forecast,24hr_general_relative_humidity_low,24hr_general_relative_humidity_high,24hr_general_temperature_low,24hr_general_temperature_high,24hr_general_wind_speed_low,24hr_general_wind_speed_high,24hr_general_wind_direction,...,24hr_period_3_forecast_south,24hr_period_3_forecast_north,call_timestamp,24hr_period_4_start,24hr_period_4_end,24hr_period_4_forecast_west,24hr_period_4_forecast_east,24hr_period_4_forecast_central,24hr_period_4_forecast_south,24hr_period_4_forecast_north
0,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T223549,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T223627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T225031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T225533,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T230035,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
forecast_2h = get_csv("forecast-2HR.csv", weather_dir)
forecast_2h.head()

,2hr_forecast_area,2hr_forecast_area_loc,2hr_forecast_value,2hr_start,2hr_end,call_timestamp
0,Ang Mo Kio,"(1.375, 103.839)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
1,Bedok,"(1.321, 103.924)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
2,Bishan,"(1.350772, 103.839)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
3,Boon Lay,"(1.304, 103.701)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
4,Bukit Batok,"(1.353, 103.754)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549


In [13]:
bing_data = get_csv("all-congestion-levels_2022-04-05T004806.csv", bing_dir)
bing_data.head()

,camera_id,direction,trafficCongestion,travelDistance,travelDuration,travelDurationTraffic,distanceUnit,durationUnit,call_timestamp,cam_loc,remarks,dir_start,dir_finish
0,1001,1,None,0.100,4,3,Kilometer,Second,2022-04-05T004806,"(1.29531332, 103.871146)",TRUE (left),"(1.2952719096725678, 103.87068089160921)","(1.2954046383155724, 103.87157229738281)"
1,1001,2,None,0.099,4,3,Kilometer,Second,2022-04-05T004806,"(1.29531332, 103.871146)",TRUE (left),"(1.295216922089859, 103.87161591936747)","(1.29509936242626, 103.87072261698584)"
2,1002,1,None,0.099,5,5,Kilometer,Second,2022-04-05T004806,"(1.319541067, 103.8785627)",TRUE (left),"(1.319649819147653, 103.87894320461515)","(1.3195731538491529, 103.87805455346877)"
3,1002,2,Medium,0.100,4,5,Kilometer,Second,2022-04-05T004806,"(1.319541067, 103.8785627)",TRUE (left),"(1.319965499763606, 103.8780229770321)","(1.320015106713897, 103.8789296718566)"
4,1003,1,None,0.105,6,3,Kilometer,Second,2022-04-05T004806,"(1.323957439, 103.8728576)",SPECIAL (5 lanes),"(1.3243485357876017, 103.87259422178255)","(1.3236450201732457, 103.87318289678052)"


In [14]:
bing_key_table = get_csv("route_data.csv", bing_dir)
bing_key_table.head()

,camera_id,direction,cam_loc,remarks,dir_start,dir_finish
0,1001,1,"(1.29531332, 103.871146)",TRUE (left),"(1.2952719096725678, 103.87068089160921)","(1.2954046383155724, 103.87157229738281)"
1,1001,2,"(1.29531332, 103.871146)",TRUE (left),"(1.295216922089859, 103.87161591936747)","(1.29509936242626, 103.87072261698584)"
2,1002,1,"(1.319541067, 103.8785627)",TRUE (left),"(1.319649819147653, 103.87894320461515)","(1.3195731538491529, 103.87805455346877)"
3,1002,2,"(1.319541067, 103.8785627)",TRUE (left),"(1.319965499763606, 103.8780229770321)","(1.320015106713897, 103.8789296718566)"
4,1003,1,"(1.323957439, 103.8728576)",SPECIAL (5 lanes),"(1.3243485357876017, 103.87259422178255)","(1.3236450201732457, 103.87318289678052)"


# Processing 

So the objective here is to make the dataset into a useable one - ie the whole long row as we would input into a model   
Chances are going to have to process individually.  
The superkey to join all tables will be the timestamp except forecast idk how that will work  

## Realtime Rainfall 

In [15]:
# original view
rainfall_realtime.head()

,timestamp,station_id,station_loc,station_name,rainfall_realtime,call_timestamp
0,2022-03-31T222500,S77,"(1.2937, 103.8125)",Alexandra Road,0.0,2022-03-31T223549
1,2022-03-31T222500,S109,"(1.3764, 103.8492)",Ang Mo Kio Avenue 5,0.0,2022-03-31T223549
2,2022-03-31T222500,S90,"(1.3191, 103.8191)",Bukit Timah Road,0.0,2022-03-31T223549
3,2022-03-31T222500,S114,"(1.38, 103.73)",Choa Chu Kang Avenue 4,0.0,2022-03-31T223549
4,2022-03-31T222500,S50,"(1.3337, 103.7768)",Clementi Road,0.0,2022-03-31T223549


In [16]:
rainfall_realtime = rainfall_realtime[["call_timestamp", "station_id", "rainfall_realtime"]]
rainfall_realtime = rainfall_realtime.rename(columns = {"station_id" : "rainfall_station_id"})
rainfall_realtime.head()

,call_timestamp,rainfall_station_id,rainfall_realtime
0,2022-03-31T223549,S77,0.0
1,2022-03-31T223549,S109,0.0
2,2022-03-31T223549,S90,0.0
3,2022-03-31T223549,S114,0.0
4,2022-03-31T223549,S50,0.0


## Realtime non rainfall

In [17]:
non_rainfall_realtime.head()

,timestamp,station_id,station_loc,station_name,wind_speed_realtime,wind_dir_realtime,RH%_realtime,air_temp_realtime,call_timestamp
0,2022-03-31T223000,S107,"(1.3135, 103.9625)",East Coast Parkway,2.7,251,78.2,28.6,2022-03-31T223549
1,2022-03-31T223000,S108,"(1.2799, 103.8703)",Marina Gardens Drive,1.6,251,93.9,27.6,2022-03-31T223549
2,2022-03-31T223000,S44,"(1.34583, 103.68166)",Nanyang Avenue,1.1,230,82.7,27.4,2022-03-31T223549
3,2022-03-31T223000,S106,"(1.4168, 103.9673)",Pulau Ubin,0.9,266,96.5,25.7,2022-03-31T223549
4,2022-03-31T223000,S24,"(1.3678, 103.9826)",Upper Changi Road North,1.8,329,84.7,27.5,2022-03-31T223549


In [18]:
non_rainfall_realtime = non_rainfall_realtime.rename(columns = {"RH%_realtime" : "humidity_realtime", 
                                                               "station_id" : "non_rainfall_station_id"})

In [19]:
non_rainfall_realtime = non_rainfall_realtime[["call_timestamp", "non_rainfall_station_id", "wind_speed_realtime", 
                                               "wind_dir_realtime", "humidity_realtime", "air_temp_realtime"]]
non_rainfall_realtime.head()

,call_timestamp,non_rainfall_station_id,wind_speed_realtime,wind_dir_realtime,humidity_realtime,air_temp_realtime
0,2022-03-31T223549,S107,2.7,251,78.2,28.6
1,2022-03-31T223549,S108,1.6,251,93.9,27.6
2,2022-03-31T223549,S44,1.1,230,82.7,27.4
3,2022-03-31T223549,S106,0.9,266,96.5,25.7
4,2022-03-31T223549,S24,1.8,329,84.7,27.5


## 4 Day Forecast

In [20]:
forecast_4d["4day_date"] = pd.to_datetime(forecast_4d["4day_date"])
forecast_4d["grp_row_num"] = forecast_4d.sort_values("4day_date").groupby(["4day_update_timestamp", "call_timestamp"])["4day_date"].cumcount()+1
forecast_4d["grp_row_num"] = forecast_4d.grp_row_num.apply(lambda x : str(x))
forecast_4d = forecast_4d.drop("4day_date", axis = 1)
forecast_4d = forecast_4d.pivot(index = "call_timestamp", columns = "grp_row_num")
forecast_4d.columns = ['_'.join(col) for col in forecast_4d.columns]
forecast_4d = forecast_4d.drop(["4day_update_timestamp_1", "4day_update_timestamp_2", 
                  "4day_update_timestamp_3", "4day_update_timestamp_4"], axis = 1)
forecast_4d = forecast_4d.reset_index()
forecast_4d.head()

,call_timestamp,4day_forecast_1,4day_forecast_2,4day_forecast_3,4day_forecast_4,4day_temperature_low_1,4day_temperature_low_2,4day_temperature_low_3,4day_temperature_low_4,4day_temperature_high_1,...,4day_wind_speed_low_3,4day_wind_speed_low_4,4day_wind_speed_high_1,4day_wind_speed_high_2,4day_wind_speed_high_3,4day_wind_speed_high_4,4day_wind_direction_1,4day_wind_direction_2,4day_wind_direction_3,4day_wind_direction_4
0,2022-03-31T182424,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,24,24,24,24,34,...,5,5,15,15,15,15,NW,NW,W,W
1,2022-03-31T182939,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,24,24,24,24,34,...,5,5,15,15,15,15,NW,NW,W,W
2,2022-03-31T183526,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,24,24,24,24,34,...,5,5,15,15,15,15,NW,NW,W,W
3,2022-03-31T184049,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,24,24,24,24,34,...,5,5,15,15,15,15,NW,NW,W,W
4,2022-03-31T184625,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,Afternoon thundery showers,24,24,24,24,34,...,5,5,15,15,15,15,NW,NW,W,W


## 24 Hour Forecast

No processing needed. But got NA to drop 

In [21]:
forecast_24h.head()

,24hr_start,24hr_end,24hr_general_forecast,24hr_general_relative_humidity_low,24hr_general_relative_humidity_high,24hr_general_temperature_low,24hr_general_temperature_high,24hr_general_wind_speed_low,24hr_general_wind_speed_high,24hr_general_wind_direction,...,24hr_period_3_forecast_south,24hr_period_3_forecast_north,call_timestamp,24hr_period_4_start,24hr_period_4_end,24hr_period_4_forecast_west,24hr_period_4_forecast_east,24hr_period_4_forecast_central,24hr_period_4_forecast_south,24hr_period_4_forecast_north
0,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T223549,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T223627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T225031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T225533,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,Thundery Showers,Thundery Showers,2022-03-31T230035,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
forecast_24h.isnull().any()

24hr_start                             False
24hr_end                               False
24hr_general_forecast                  False
24hr_general_relative_humidity_low     False
24hr_general_relative_humidity_high    False
24hr_general_temperature_low           False
24hr_general_temperature_high          False
24hr_general_wind_speed_low            False
24hr_general_wind_speed_high           False
24hr_general_wind_direction            False
24hr_period_1_start                    False
24hr_period_1_end                      False
24hr_period_1_forecast_west            False
24hr_period_1_forecast_east            False
24hr_period_1_forecast_central         False
24hr_period_1_forecast_south           False
24hr_period_1_forecast_north           False
24hr_period_2_start                    False
24hr_period_2_end                      False
24hr_period_2_forecast_west            False
24hr_period_2_forecast_east            False
24hr_period_2_forecast_central         False
24hr_perio

In [23]:
forecast_24h = forecast_24h.drop(["24hr_period_4_start", "24hr_period_4_end", "24hr_period_4_forecast_west", 
                                 "24hr_period_4_forecast_east", "24hr_period_4_forecast_central", 
                                 "24hr_period_4_forecast_south", "24hr_period_4_forecast_north"], axis = 1)

In [24]:
long_df1 = pd.melt(forecast_24h, id_vars = ["call_timestamp", "24hr_period_1_start", "24hr_period_1_end"],
        value_vars= ["24hr_period_1_forecast_west", 
                                 "24hr_period_1_forecast_east", "24hr_period_1_forecast_central", 
                                 "24hr_period_1_forecast_south", "24hr_period_1_forecast_north"])

long_df2 = pd.melt(forecast_24h, id_vars = ["call_timestamp", "24hr_period_2_start", "24hr_period_2_end"],
        value_vars= ["24hr_period_2_forecast_west", 
                                 "24hr_period_2_forecast_east", "24hr_period_2_forecast_central", 
                                 "24hr_period_2_forecast_south", "24hr_period_2_forecast_north"])

long_df3 = pd.melt(forecast_24h, id_vars = ["call_timestamp", "24hr_period_3_start", "24hr_period_3_end"],
        value_vars= ["24hr_period_3_forecast_west", 
                                 "24hr_period_3_forecast_east", "24hr_period_3_forecast_central", 
                                 "24hr_period_3_forecast_south", "24hr_period_3_forecast_north"])

In [25]:
long_df1["compass"] = long_df1.variable.apply(lambda x : x.split("_")[-1])
long_df2["compass"] = long_df2.variable.apply(lambda x : x.split("_")[-1])
long_df3["compass"] = long_df3.variable.apply(lambda x : x.split("_")[-1])

In [26]:
long_df1 = long_df1.drop("variable", axis = 1).rename(columns = {"value" : "24hr_period_1"})
long_df2 = long_df2.drop("variable", axis = 1).rename(columns = {"value" : "24hr_period_2"})
long_df3 = long_df3.drop("variable", axis = 1).rename(columns = {"value" : "24hr_period_3"})

In [27]:
forecast_24h = forecast_24h.drop(["24hr_period_1_forecast_west", 
                                 "24hr_period_1_forecast_east", "24hr_period_1_forecast_central", 
                                 "24hr_period_1_forecast_south", "24hr_period_1_forecast_north", 
                                 "24hr_period_2_forecast_west", 
                                 "24hr_period_2_forecast_east", "24hr_period_2_forecast_central", 
                                 "24hr_period_2_forecast_south", "24hr_period_2_forecast_north", 
                                 "24hr_period_3_forecast_west", 
                                 "24hr_period_3_forecast_east", "24hr_period_3_forecast_central", 
                                 "24hr_period_3_forecast_south", "24hr_period_3_forecast_north", 
                                 "24hr_period_1_start", "24hr_period_1_end", 
                                 "24hr_period_2_start", "24hr_period_2_end", 
                                 "24hr_period_3_start", "24hr_period_3_end"], axis = 1)

In [28]:
forecast_24h = forecast_24h\
.merge(long_df1, how = "inner", on = "call_timestamp")\
.merge(long_df2, how = "inner", on = ["call_timestamp", "compass"])\
.merge(long_df3, how = "inner", on = ["call_timestamp", "compass"])

In [29]:
forecast_24h.head()

,24hr_start,24hr_end,24hr_general_forecast,24hr_general_relative_humidity_low,24hr_general_relative_humidity_high,24hr_general_temperature_low,24hr_general_temperature_high,24hr_general_wind_speed_low,24hr_general_wind_speed_high,24hr_general_wind_direction,...,24hr_period_1_start,24hr_period_1_end,24hr_period_1,compass,24hr_period_2_start,24hr_period_2_end,24hr_period_2,24hr_period_3_start,24hr_period_3_end,24hr_period_3
0,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,2022-03-31T180000,2022-04-01T060000,Cloudy,west,2022-04-01T060000,2022-04-01T120000,Partly Cloudy (Day),2022-04-01T120000,2022-04-01T180000,Thundery Showers
1,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,2022-03-31T180000,2022-04-01T060000,Cloudy,east,2022-04-01T060000,2022-04-01T120000,Partly Cloudy (Day),2022-04-01T120000,2022-04-01T180000,Thundery Showers
2,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,2022-03-31T180000,2022-04-01T060000,Cloudy,central,2022-04-01T060000,2022-04-01T120000,Partly Cloudy (Day),2022-04-01T120000,2022-04-01T180000,Thundery Showers
3,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,2022-03-31T180000,2022-04-01T060000,Cloudy,south,2022-04-01T060000,2022-04-01T120000,Partly Cloudy (Day),2022-04-01T120000,2022-04-01T180000,Thundery Showers
4,2022-03-31T180000,2022-04-01T180000,Thundery Showers,60,95,24,34,10,20,NW,...,2022-03-31T180000,2022-04-01T060000,Cloudy,north,2022-04-01T060000,2022-04-01T120000,Partly Cloudy (Day),2022-04-01T120000,2022-04-01T180000,Thundery Showers


## 2 Hour Forecast

In [30]:
forecast_2h.head()

,2hr_forecast_area,2hr_forecast_area_loc,2hr_forecast_value,2hr_start,2hr_end,call_timestamp
0,Ang Mo Kio,"(1.375, 103.839)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
1,Bedok,"(1.321, 103.924)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
2,Bishan,"(1.350772, 103.839)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
3,Boon Lay,"(1.304, 103.701)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549
4,Bukit Batok,"(1.353, 103.754)",Cloudy,2022-03-31T220000,2022-04-01T000000,2022-03-31T223549


In [31]:
forecast_2h = forecast_2h.drop(["2hr_forecast_area_loc", "2hr_start", "2hr_end"], axis = 1)
forecast_2h.head()

,2hr_forecast_area,2hr_forecast_value,call_timestamp
0,Ang Mo Kio,Cloudy,2022-03-31T223549
1,Bedok,Cloudy,2022-03-31T223549
2,Bishan,Cloudy,2022-03-31T223549
3,Boon Lay,Cloudy,2022-03-31T223549
4,Bukit Batok,Cloudy,2022-03-31T223549


## Bing Data 

In [32]:
bing_key_table = bing_key_table[["camera_id", "direction"]].rename(columns = {"camera_id" : "cam_id"})

In [33]:
bing_data = bing_data[["camera_id", "direction", "call_timestamp", "trafficCongestion",
                      "travelDistance", "travelDuration", "travelDurationTraffic"]]\
.rename(columns = {"camera_id" : "cam_id"})

In [34]:
bing_data["ratio"] = bing_data.travelDurationTraffic / bing_data.travelDuration

In [35]:
bing_data.head()

,cam_id,direction,call_timestamp,trafficCongestion,travelDistance,travelDuration,travelDurationTraffic,ratio
0,1001,1,2022-04-05T004806,None,0.100,4,3,0.75
1,1001,2,2022-04-05T004806,None,0.099,4,3,0.75
2,1002,1,2022-04-05T004806,None,0.099,5,5,1.00
3,1002,2,2022-04-05T004806,Medium,0.100,4,5,1.25
4,1003,1,2022-04-05T004806,None,0.105,6,3,0.50


In [36]:
bing_data[["trafficCongestion", "ratio"]].groupby("trafficCongestion").agg([min, max, np.mean])

ratio                    
                        min       max      mean
trafficCongestion                              
Heavy              1.583333  6.375000  2.961806
Medium             1.250000  1.333333  1.257937
Mild               1.100000  1.200000  1.182684
None               0.305085  1.090909  0.914799
Unknown                 NaN       NaN       NaN

In [37]:
bing_data[["trafficCongestion", "cam_id"]].groupby("trafficCongestion").count() # need to look out for unknowns 

,cam_id
trafficCongestion,
Heavy,4
Medium,35
Mild,11
None,128
Unknown,1


So basically clear case of correlated variables 

In [38]:
bing_data = bing_data[["cam_id", "direction", "call_timestamp", "trafficCongestion"]]
##bing_data = bing_data[bing_data.trafficCongestion != "Unknown"]

# Combining Tables 

Logic would be to first gather all the time that EVERY dataset has.  
This is to allow for it to be able to join all table. 
Also need to expand the camera mapping 

In [39]:
cam_all = cam_to_loc.merge(bing_key_table, on = "cam_id")

In [40]:
# Based on shimar, the time should all be consistent 
# But this is done just for precaution

all_time = set(rainfall_realtime.call_timestamp)\
.intersection(set(non_rainfall_realtime.call_timestamp))\
.intersection(set(forecast_4d.call_timestamp))\
.intersection(set(forecast_24h.call_timestamp))\
.intersection(set(forecast_2h.call_timestamp))#\
#.intersection(set(bing_data.call_timestamp))

In [41]:
all_time = pd.DataFrame(all_time, columns = ["call_timestamp"])

Next step is to cross join this with the camera.  
All cameras should have entries for all time. 

In [42]:
base_df = all_time.merge(cam_all, how = "cross") # change to cam_all

In [43]:
base_df

,call_timestamp,cam_id,non_rainfall_station_id,rainfall_station_id,2hr_forecast_area,compass,direction
0,2022-04-04T002753,1001,S108,S119,Kallang,south,1
1,2022-04-04T002753,1001,S108,S119,Kallang,south,2
2,2022-04-04T002753,1002,S108,S215,Geylang,east,1
3,2022-04-04T002753,1002,S108,S215,Geylang,east,2
4,2022-04-04T002753,1003,S108,S123,Kallang,central,1
...,...,...,...,...,...,...,...
226072,2022-04-04T224542,9704,S44,S40,Mandai,north,2
226073,2022-04-04T224542,9705,S44,S104,Mandai,north,1
226074,2022-04-04T224542,9705,S44,S104,Mandai,north,2
226075,2022-04-04T224542,9706,S44,S40,Mandai,north,1


In [44]:
df = base_df.merge(rainfall_realtime, on = ["call_timestamp", "rainfall_station_id"], how = "left")

In [45]:
len(df) == len(base_df)

True

In [46]:
df2 = df.merge(non_rainfall_realtime, on = ["call_timestamp", "non_rainfall_station_id"], how = "left")

In [47]:
len(df2) == len(base_df)

True

In [48]:
df3 = df2.merge(forecast_2h, on = ["call_timestamp", "2hr_forecast_area"], how = "left")

In [49]:
len(df3) == len(base_df)

True

In [50]:
df4 = df3.merge(forecast_4d, on = ["call_timestamp"])

In [51]:
len(df4) == len(base_df)

True

In [52]:
df5 = df4.merge(forecast_24h, on = ["call_timestamp", "compass"])

In [53]:
len(df5) == len(base_df)

True

In [54]:
df6 = df5.merge(bing_data, on = ["call_timestamp", "cam_id", "direction"])

In [55]:
len(df6) == len(base_df)

False